In [1]:
%load_ext autoreload
%autoreload 2

from FIREreader import FIREreader
import numpy as np
import os
import h5py

In [2]:
## initialize reader object and choose simulation to run
reader = FIREreader()
reader.directory = "/Users/agurvich/research/snaps/CHIMES/output"
reader.snapnum = 500
## could read this from snapshot times
current_redshift=0

In [3]:
## decide which part types to save to JSON
reader.returnParts = ['PartType0', 'PartType4']

## choose the names the particle types will get in the UI
reader.names = {'PartType0':'Gas', 
                  'PartType1':'HRDM', 
                  'PartType2':'LRDM', 
                  'PartType4':'Stars' }

In [4]:
#define the defaults; this must be run first if you want to change the defaults below
reader.defineDefaults()

## by what factor should you sub-sample the data (e.g. array[::decimate])
decimate = [10, 10.]

In [5]:
## load in the data from hdf5 files and put it into reader.partsDict
for i,p in enumerate(reader.returnParts):
    reader.decimate[p] = decimate[i]
    reader.returnKeys[p] = ['Coordinates', 'Density','Velocities','HIIAbundance','Temperature','AgeGyr']
    #Note: you should only try to filter on scalar values (like density).  
    #The magnitude of the Velocities are calculated in Firefly, and you will automatically be allowed to filter on it
    reader.addFilter[p] = [False, True, False,True,True,True]
    ## tell it to do the log of density when filtering
    reader.dolog[p] = [False, True, False,False,True,False]
    
    
    #NOTE: all dictionaries in the "options" reference the swapped names (i.e., reader.names) you define above.  
    #If you don't define reader.names, then you can use the default keys from the hdf5 files 
    #(but then you will see those hdf5 names in the Firefly GUI)
    pp = reader.names[p]
    ## set the initial size of the particles when the interface loads
    reader.options['sizeMult'][pp] = 0.3
    
## set the default colors when the interface loads
reader.options['color'] = {'Gas':  [1., 0., 0., 1.],  
                           'HRDM': [1., 1., 0., 0.1],  
                           'LRDM': [1., 1., 0., 0.1],  
                           'Stars':[0., 0., 1., 0.1]} 

## set the camera center to be at the origin (defaults to np.mean(Coordinates) otherwise)
##     later on we subtract out halo_center from coordinates but could instead make this halo_center
reader.options['center'] = np.array([0., 0., 0.])

## initialize filter flags and options
reader.defineFilterKeys()
## load in return keys from snapshot
filenames_opened = reader.populate_dict()

{'PartType4': ['Coordinates', 'Density', 'Velocities', 'HIIAbundance', 'Temperature', 'AgeGyr'], 'PartType0': ['Coordinates', 'Density', 'Velocities', 'HIIAbundance', 'Temperature', 'AgeGyr']}
/Users/agurvich/research/snaps/CHIMES/output/snapshot_500.hdf5
PartType0 has no AgeGyr
/Users/agurvich/research/snaps/CHIMES/output/snapshot_500.hdf5
PartType4 has no Density
PartType4 has no HIIAbundance
PartType4 has no Temperature


In [6]:
## calculate the radius from the halo center
gas_radii = np.sum(reader.partsDict['PartType0']['Coordinates']**2,axis=1)**0.5
star_radii = np.sum(reader.partsDict['PartType4']['Coordinates']**2,axis=1)**0.5

## add new radius array to the dictionary using addtodict method
reader.addtodict(reader.partsDict,None,'PartType0','Radius',0,0,vals=gas_radii, filterFlag = True)
reader.addtodict(reader.partsDict,None,'PartType4','Radius',0,0,vals=star_radii, filterFlag = True)

In [7]:
# Code mass -> g , (code length)^-3 -> cm^-3 , g -> nHydrogen
DENSITYFACT=2e43*(3.086e21)**-3/(1.67e-24)
reader.partsDict['PartType0']['log10Density'] = reader.partsDict['PartType0']['log10Density']+np.log10(DENSITYFACT)

In [8]:
## finish up, let's shuffle + decimate, add the GUI friendly names, and create our final JSON!
reader.shuffle_dict()
reader.swap_dict_names()
reader.createJSON()

decimating and shuffling ...
decimating and shuffling ...
('dataDir', None)
writing JSON files ...
Gas
Stars
CHIMES_500/FIREdataOptions.json
